In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from google.colab.patches import cv2_imshow



In [ ]:
class ArthritisPoseCoach:
    def __init__(self):
        # MediaPipe Pose Estimation
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # Exercise recommendations
        self.exercise_db = {
            'low_mobility': [
                "Seated stretches",
                "Gentle arm circles",
                "Ankle rotations"
            ],
            'medium_mobility': [
                "Resistance band exercises",
                "Walking in place",
                "Lightweight lifting"
            ],
            'high_mobility': [
                "Yoga adaptations",
                "Swimming exercises",
                "Balance training"
            ]
        }


In [ ]:

    def analyze_pose(self, frame):
        """Analyze body pose and calculate mobility score"""
        # Convert BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process frame
        results = self.pose.process(rgb_frame)

        # Draw landmarks on the frame
        annotated_frame = frame.copy()

        if results.pose_landmarks:
            # Draw landmarks
            mp.solutions.drawing_utils.draw_landmarks(
                annotated_frame,
                results.pose_landmarks,
                self.mp_pose.POSE_CONNECTIONS
            )

            # Angle calculation function
            def calculate_angle(a, b, c):
                a = np.array([a.x, a.y])
                b = np.array([b.x, b.y])
                c = np.array([c.x, c.y])

                radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)

                return angle if angle <= 180 else 360 - angle

            # Get landmarks
            landmarks = results.pose_landmarks.landmark

            # Calculate mobility metrics
            mobility_metrics = {
                'shoulder_mobility': calculate_angle(
                    landmarks[self.mp_pose.PoseLandmarks.LEFT_SHOULDER],
                    landmarks[self.mp_pose.PoseLandmarks.LEFT_ELBOW],
                    landmarks[self.mp_pose.PoseLandmarks.LEFT_WRIST]
                ),
                'hip_mobility': calculate_angle(
                    landmarks[self.mp_pose.PoseLandmarks.LEFT_HIP],
                    landmarks[self.mp_pose.PoseLandmarks.LEFT_KNEE],
                    landmarks[self.mp_pose.PoseLandmarks.LEFT_ANKLE]
                )
            }



In [ ]:
            # Determine mobility level
            avg_mobility = sum(mobility_metrics.values()) / len(mobility_metrics)

            mobility_level = (
                'low_mobility' if avg_mobility < 60 else
                'medium_mobility' if avg_mobility < 120 else
                'high_mobility'
            )

            return {
                'mobility_score': avg_mobility,
                'mobility_level': mobility_level,
                'recommended_exercises': self.exercise_db[mobility_level],
                'frame': annotated_frame
            }




NameError: name 'mobility_metrics' is not defined

In [ ]:
    def start_pose_tracking(self, video_path):
        """Pose tracking for Colab"""
        # Open video capture
        cap = cv2.VideoCapture(video_path)

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            # Analyze pose
            pose_result = self.analyze_pose(frame)

            if pose_result:
                # Annotate frame with results
                result_frame = pose_result['frame']
                cv2.putText(
                    result_frame,
                    f"Mobility: {pose_result['mobility_level']}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2
                )

                # Display frame
                cv2_imshow(result_frame)

                # Print exercise recommendations
                print("Recommended Exercises:", pose_result['recommended_exercises'])

        cap.release()


In [ ]:

def main():

    coach = ArthritisPoseCoach()
    coach.start_pose_tracking('/content/video.mp4')

if __name__ == "__main__":
    main()